<a href="https://colab.research.google.com/github/deanhoperobertson/Named-Enitty-Recognition/blob/master/Models/Hyperparameter%20Optimization/BiLSTM-CRF/SENNA/BiLSTM-CRF%20SENNA%20(Optimizer).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo pip install git+https://www.github.com/keras-team/keras-contrib.git
!pip install sklearn_crfsuite
!pip install keras==2.2.4
%tensorflow_version 1.x
#!pip install tensorflow-gpu==1.15.0

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-cacosru_
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-cacosru_
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101064 sha256=1d0015d47f607855dd1f943ba426cd1a9157111138f8874f76291fc04be58a1b
  Stored in directory: /tmp/pip-ephem-wheel-cache-ro2yy6ur/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib
TensorFlow 1.x selected.


In [ ]:
import pandas as pd
import re
import numpy as np
import urllib.request
from sklearn.model_selection import train_test_split

#keras and tensorflow packages
import tensorflow as tf
from keras.layers.merge import add
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy
from keras_contrib.metrics import crf_accuracy

#evaluation
from sklearn_crfsuite.metrics import flat_classification_report,flat_f1_score,flat_precision_score
print(tf.__version__)

1.15.2


Using TensorFlow backend.


In [ ]:
#Ensure Reproducibility
import numpy as np
import random as rn
import os

random_seed=3

# 1) set numpy random seed
np.random.seed(random_seed)
# 2) set build-in random seed
rn.seed(random_seed)
# 3) set PYTHONHASHSEED
os.environ['PYTHONHASHSEED']=str(random_seed)
# 4) set tensorflow random seed
tf.random.set_random_seed(random_seed)

# 5) Force TensorFlow to use single thread.
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
from keras import backend as K
tf.set_random_seed(1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
#import data from my github repo
train_url = "https://raw.githubusercontent.com/deanhoperobertson/Named-Enitty-Recognition/master/Data/train.txt"
test_url = "https://raw.githubusercontent.com/deanhoperobertson/Named-Enitty-Recognition/master/Data/test.txt"
train = urllib.request.urlopen(train_url).read()
test = urllib.request.urlopen(test_url).read()
train = train.decode('utf-8')
test = test.decode('utf-8')

def readstring(filename, meth):
    f = filename.split('\n')
    sentences = []
    sentence = []
    for line in f:
        if len(line) == 0 or line.startswith('-DOCSTART') or line[0] == "\n":
            if len(sentence) > 0:
                sentences.append(sentence)
                sentence = []
            continue
        splits = line.split(' ')
        if meth.lower()=="numbers1":
            sentence.append([hasNumbers1(splits[0]), splits[-1].strip()])
        elif meth.lower()=="numbers2":
            sentence.append([hasNumbers2(splits[0]), splits[-1].strip()])
        else:
            sentence.append([splits[0], splits[-1].strip()])
    if len(sentence) > 0:
        sentences.append(sentence)
        sentence = []
    return sentences

def hasNumbers1(inputString):
    if re.search(r'\d', inputString):
        return "0"
    else:return(inputString)

def hasNumbers2(text):
  if text.isdigit():
      return "1"
  elif re.search(r'\d',text) and re.search(r'\,|\.',text):
      return "1" 
  else:
      if re.search(r'\d', text):
          return(re.sub('\d','D', text))
      else:
          return text

#preproces the txt file
train_data = readstring(train,"Numbers1")
test_data = readstring(test,"Numbers1")

#create corpus
corpus = train_data.copy()
corpus.extend(test_data)

In [ ]:
def reformat_data(data,meth):
  if meth.lower() == "data":
    i=0
  else: i=1
  train = []
  output= []
  for sentence in data:
    words=[]
    for x in sentence:
      words.append(x[i])
    train.append(words)

  for i in train:
    string = ' '.join(i)
    output.append(string)
  return output

def get_max_length(corpus):
  length = []
  for sentence in corpus:
    length.append(len(sentence))
  return int(max(length))

def number_of_tags(corpus):
  tags=[]
  for sentence in corpus:
    for tag in sentence:
      tags.append(tag[1])
  return int(len(list(set(tags))))


MAX_LEN = get_max_length(corpus)
N_tags = number_of_tags(corpus)

train = reformat_data(train_data,"data")
test = reformat_data(test_data,"data")

train_y = reformat_data(train_data,"tags")
test_y = reformat_data(test_data,"tags")


## Load Embeddings

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#root_path = "/content/drive/My Drive/glove.6B.50d.txt"
root_path = "/content/drive/My Drive/SENNA.txt"

embeddings_index={}
f = open(root_path, encoding = "utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 130000 word vectors.


## Data Processing

In [ ]:
def prep_data(xx_train,yy_train,xx_test,yy_test):
  
  token_word = text.Tokenizer(char_level=False, lower=True, filters="}", oov_token='UNK')
  token_word.fit_on_texts(xx_train)

  X_train = sequence.pad_sequences(token_word.texts_to_sequences(xx_train), maxlen=MAX_LEN,padding="post")
  X_test = sequence.pad_sequences(token_word.texts_to_sequences(xx_test), maxlen=MAX_LEN,padding="post")

  # create a tokenizer
  token_tag = text.Tokenizer(char_level=False, lower=False, filters="}")
  token_tag.fit_on_texts(train_y)

  # convert training labels
  Y = sequence.pad_sequences(token_tag.texts_to_sequences(yy_train), maxlen=MAX_LEN,padding="post")
  Y_train = [to_categorical(i, num_classes=N_tags+1) for i in Y]

  # convert test labels
  Y = sequence.pad_sequences(token_tag.texts_to_sequences(yy_test), maxlen=MAX_LEN,padding="post")
  Y_test = [to_categorical(i, num_classes=N_tags+1) for i in Y]

  return X_train, X_test, Y_train, Y_test, token_word

## Grid Search

In [ ]:
%%time
from sklearn.model_selection import KFold
from keras import optimizers
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=2)


kf = KFold(n_splits=3,random_state=3, shuffle=True)

grid_results = []
OPTIMIZERS =["nadam", "adam", "sgd"]
BATCH_SIZE= 64
EPOCHS=20
fold_count = 1

for OPT in OPTIMIZERS:
  for train_index, test_index in kf.split(train):
    print("FOLD Number: %d/%d" %(fold_count,len(OPTIMIZERS)*3))
    
    #split data
    xx_train,yy_train = np.array(train)[train_index], np.array(train_y)[train_index]
    xx_test,yy_test = np.array(train)[test_index], np.array(train_y)[test_index]

    X_train, X_test, Y_train, Y_test, token_word = prep_data(xx_train, yy_train,xx_test, yy_test)

    #---------create emedding matrix------------
    EMBEDDING=len(coefs)
    word_index = token_word.word_index
    embedding_matrix = np.zeros((len(token_word.word_index) + 1, EMBEDDING))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
          # words not found in embedding index will be all-zeros.
          embedding_matrix[i] = embedding_vector

    #---------Define Model Build-------------
    def create_model(opt):
      input =  Input(shape=(MAX_LEN,))
      model_2 = Embedding(len(token_word.word_index) + 1,output_dim=EMBEDDING,
                      weights=[embedding_matrix],input_length=MAX_LEN,
                      trainable=False,mask_zero=True)(input)
      model_2 = Bidirectional(LSTM(units=100, return_sequences=True,
                                recurrent_dropout=0.1, dropout=0.1))(model_2)
      model_2 = TimeDistributed(Dense(20, activation="relu"))(model_2)
      crf = CRF(N_tags+1)  # CRF layer
      out = crf(model_2)  # output
      model_2 = Model(input, out)
      
      model_2.compile(optimizer=opt, loss=crf_loss,metrics=[crf_viterbi_accuracy])
      return model_2

  #---------Fit Model-----------------
    model = create_model(OPT)
    history = model.fit(X_train, np.array(Y_train), batch_size=BATCH_SIZE, epochs=EPOCHS,verbose=1, callbacks=[early_stopping], validation_data=(X_test,np.array(Y_test)))

    grid_results.append({"train_loss":history.history["loss"][-1],
                      "train_acc": history.history["crf_viterbi_accuracy"][-1],
                      "val_loss": history.history["val_loss"][-1],
                      "val_acc": history.history["val_crf_viterbi_accuracy"][-1],
                      "epochs": len(history.history["loss"]),
                      "Optimizer": OPT,
                      })

    fold_count +=1

FOLD Number: 1/9







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 9360 samples, validate on 4681 samples
Epoch 1/20
9360/9360 [==============================] - 101s 11ms/step - loss: 39.2304 - crf_viterbi_accuracy: 0.9121 - val_loss: 39.3042 - val_crf_viterbi_accuracy: 0.9424
Epoch 2/20
9360/9360 [==============================] - 96s 10ms/step - loss: 39.0315 - crf_viterbi_accuracy: 0.9530 - val_loss: 39.2606 - val_crf_viterbi_accuracy: 0.9526
Epoch 3/20
9360/9360 [==============================] - 95s 10ms/step - loss: 38.9972 - crf_viterbi_accuracy: 0.9619 - val_loss: 39.2620 - val_crf_viterbi_accuracy: 0.9536
Epoch 4/20
9360/9360 [==============================] - 94s 10ms/step - loss: 38.9759 - crf_viterbi_accuracy: 0.9675 - val_loss: 39.2294 - val_crf_viterbi_accuracy: 0.9620
Epoch 5/20
9360/9360 [=====

## TEST

In [ ]:
df_results = pd.DataFrame(grid_results)
df_results.groupby("Optimizer").mean().sort_values(by=["val_acc"], ascending=False)

,train_loss,train_acc,val_loss,val_acc,epochs
Optimizer,,,,,
adam,39.008473,0.980577,39.068410,0.966896,14.333333
nadam,39.011718,0.979076,39.077414,0.960537,8.000000
sgd,39.267752,0.916852,39.288684,0.908768,20.000000


In [ ]:
from google.colab import files
df_results.to_csv('Optimizer_grid_senna.csv')
files.download('Optimizer_grid_senna.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>